<a href="https://colab.research.google.com/github/ahmetcangunay/Sentiment_Analysis_of_Amazon_Reviews/blob/main/Sentiment_Analysis_of_Amazon_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [179]:
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import warnings

In [180]:
%matplotlib inline

nltk.download('vader_lexicon')

init_notebook_mode(connected=True)
cf.go_offline

warnings.filterwarnings('ignore')
warnings.warn('This will not show!')

pd.set_option('display.max_columns', None)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [181]:
df = pd.read_csv('/content/drive/MyDrive/Python Projects/Sentiment Analysis of Amazon Reviews/amazon.csv', index_col=0)

In [182]:
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0
3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.0
4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.0


In [183]:
df.shape

(4915, 11)

In [184]:
df['wilson_lower_bound'].value_counts()

0.000000e+00    4496
2.065493e-01     266
9.453121e-02      46
3.423802e-01      28
6.149194e-02       9
4.385030e-01       7
4.870000e-17       6
5.101092e-01       5
5.655175e-01       5
3.006418e-01       5
2.307243e-01       4
4.092754e-01       3
3.755346e-01       3
4.868722e-01       3
2.076596e-01       3
6.456696e-01       2
2.567962e-02       1
6.703344e-01       1
6.397717e-01       1
1.268070e-01       1
9.121389e-01       1
6.096657e-01       1
7.321357e-01       1
2.999933e-01       1
7.004421e-01       1
6.635948e-01       1
9.365186e-01       1
7.846510e-01       1
1.759911e-01       1
3.475118e-02       1
4.901625e-01       1
9.575439e-01       1
3.622411e-02       1
3.589345e-01       1
1.500390e-01       1
4.364972e-01       1
8.185767e-01       1
6.574110e-01       1
5.004362e-01       1
8.081087e-01       1
Name: wilson_lower_bound, dtype: int64

In [185]:
df.sort_values('wilson_lower_bound', ascending=False, inplace=True)

In [186]:
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"If your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,Sandisk announcement of the first 128GB micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [187]:
def missing_values(df):
  na_cols = [col for col in df.columns if df[col].isnull().sum() > 0]
  n_miss = df[na_cols].isnull().sum().sort_values(ascending=True)
  miss_ratio = (df[na_cols].isnull().sum() / df.shape[0] * 100).sort_values(ascending=True)
  missing_df = pd.concat([n_miss, np.round(miss_ratio, 2)], axis=1, keys=['Missing Values', 'Ratio'])
  missing_df = pd.DataFrame(missing_df)
  return missing_df

def check_df(df, head=5, tail=5):
  print('SHAPE'.center(82, '~'))
  print("ROWS: {}".format(df.shape[0]))
  print("COLUMNS: {}".format(df.shape[1]))
  print('TYPES'.center(82, '~'))
  print(df.dtypes)
  print("".center(82, '~'))
  print(missing_values(df))
  print('DUPLICATED VALUES'.center(82, '~'))
  print(df.duplicated().sum())
  print('QUANTILES'.center(82, '~'))
  print(df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~SHAPE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ROWS: 4915
COLUMNS: 11
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~TYPES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
              Missing Values  Ratio
reviewerName               1   0.02
reviewText                 1   0.02
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~DUPLICATED VALUES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~QUANTILES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       0.00  0.05   0.50        0.95       0.99         1.00
overall  

In [188]:
def check_class(dataframe):
  n_unique_df = pd.DataFrame({'Variable': dataframe.columns,
                              'Classes': [dataframe[i].nunique() for i in dataframe.columns]})
  n_unique_df = n_unique_df.sort_values('Classes', ascending=False)
  n_unique_df = n_unique_df.reset_index(drop=True)
  return n_unique_df

check_class(df)

,Variable,Classes
0,reviewText,4912
1,reviewerName,4594
2,reviewTime,690
3,day_diff,690
4,wilson_lower_bound,40
5,score_average_rating,28
6,score_pos_neg_diff,27
7,total_vote,26
8,helpful_yes,23
9,helpful_no,17


In [189]:
constraints = ['#B32D22', '#EBE00C', '#1FEB0C', '#0C92EB', '#EB0CD5']

def categorical_variable_summmary(df, col_name):
  fig = make_subplots(rows=1, cols=2, subplot_titles=('Counplot', 'Percentage'), specs=[[{'type':'xy'},{'type':'domain'}]])

  fig.add_trace(go.Bar(y=df[col_name].value_counts().values.tolist(),
                       x=[str(i) for i in df[col_name].value_counts().index],
                       text=df[col_name].value_counts().values.tolist(),
                       textfont=dict(size=14), name=col_name,
                       textposition='auto', showlegend=False,
                       marker=dict(color=constraints,
                                   line=dict(color='#DBE6EC', width=1))),
                row=1, col=1)
  
  fig.add_trace(go.Pie(labels=df[col_name].value_counts().keys(),
                       values=df[col_name].value_counts().values,
                       textfont=dict(size=18),
                       textposition='auto',
                       showlegend=False,
                       name=col_name,
                       marker=dict(colors=constraints)),
                row=1, col=2)
  fig.update_layout(title={'text':col_name,
                           'y':0.9,
                           'x':0.5,
                           'xanchor':'center',
                           'yanchor':'top'},
                    template='plotly_white')
  fig.show(renderer="colab")
  #iplot(fig)

In [190]:
categorical_variable_summmary(df, 'overall')

In [191]:
df.reviewText.head()

2031    [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...
3449    I have tested dozens of SDHC and micro-SDHC ca...
4212    NOTE:  please read the last update (scroll to ...
317     If your card gets hot enough to be painful, it...
4672    Sandisk announcement of the first 128GB micro ...
Name: reviewText, dtype: object

In [192]:
review_example = df.reviewText[2031]
review_example

'[[ UPDATE - 6/19/2014 ]]So my lovely wife bought me a Samsung Galaxy Tab 4 for Father\'s Day and I\'ve been loving it ever since.  Just as other with Samsung products, the Galaxy Tab 4 has the ability to add a microSD card to expand the memory on the device.  Since it\'s been over a year, I decided to do some more research to see if SanDisk offered anything new.  As of 6/19/2014, their product lineup for microSD cards from worst to best (performance-wise) are the as follows:SanDiskSanDisk UltraSanDisk Ultra PLUSSanDisk ExtremeSanDisk Extreme PLUSSanDisk Extreme PRONow, the difference between all of these cards are simply the speed in which you can read/write data to the card.  Yes, the published rating of most all these cards (except the SanDisk regular) are Class 10/UHS-I but that\'s just a rating... Actual real world performance does get better with each model, but with faster cards come more expensive prices.  Since Amazon doesn\'t carry the Ultra PLUS model of microSD card, I had 

In [193]:
review_example = re.sub('[^a-zA-Z]',' ', review_example)
review_example

'   UPDATE               So my lovely wife bought me a Samsung Galaxy Tab   for Father s Day and I ve been loving it ever since   Just as other with Samsung products  the Galaxy Tab   has the ability to add a microSD card to expand the memory on the device   Since it s been over a year  I decided to do some more research to see if SanDisk offered anything new   As of            their product lineup for microSD cards from worst to best  performance wise  are the as follows SanDiskSanDisk UltraSanDisk Ultra PLUSSanDisk ExtremeSanDisk Extreme PLUSSanDisk Extreme PRONow  the difference between all of these cards are simply the speed in which you can read write data to the card   Yes  the published rating of most all these cards  except the SanDisk regular  are Class    UHS I but that s just a rating    Actual real world performance does get better with each model  but with faster cards come more expensive prices   Since Amazon doesn t carry the Ultra PLUS model of microSD card  I had to do

In [194]:
review_example = review_example.lower().split()
review_example

['update',
 'so',
 'my',
 'lovely',
 'wife',
 'bought',
 'me',
 'a',
 'samsung',
 'galaxy',
 'tab',
 'for',
 'father',
 's',
 'day',
 'and',
 'i',
 've',
 'been',
 'loving',
 'it',
 'ever',
 'since',
 'just',
 'as',
 'other',
 'with',
 'samsung',
 'products',
 'the',
 'galaxy',
 'tab',
 'has',
 'the',
 'ability',
 'to',
 'add',
 'a',
 'microsd',
 'card',
 'to',
 'expand',
 'the',
 'memory',
 'on',
 'the',
 'device',
 'since',
 'it',
 's',
 'been',
 'over',
 'a',
 'year',
 'i',
 'decided',
 'to',
 'do',
 'some',
 'more',
 'research',
 'to',
 'see',
 'if',
 'sandisk',
 'offered',
 'anything',
 'new',
 'as',
 'of',
 'their',
 'product',
 'lineup',
 'for',
 'microsd',
 'cards',
 'from',
 'worst',
 'to',
 'best',
 'performance',
 'wise',
 'are',
 'the',
 'as',
 'follows',
 'sandisksandisk',
 'ultrasandisk',
 'ultra',
 'plussandisk',
 'extremesandisk',
 'extreme',
 'plussandisk',
 'extreme',
 'pronow',
 'the',
 'difference',
 'between',
 'all',
 'of',
 'these',
 'cards',
 'are',
 'simply',
 

In [195]:
rt = lambda x: re.sub("[^a-zA-Z]", ' ', str(x))
df["reviewText"] = df["reviewText"].map(rt)
df["reviewText"] = df["reviewText"].str.lower()
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [196]:
df[['polarity', 'subjectivity']] = df['reviewText'].apply(lambda Text:pd.Series(TextBlob(Text).sentiment))

for index, row in df['reviewText'].iteritems():

  score = SentimentIntensityAnalyzer().polarity_scores(row)

  neg = score['neg']
  neu = score['neu']
  pos = score['pos']

  if neg > pos:
    df.loc[index, 'sentiment'] = "Negative"
  elif pos > neg:
    df.loc[index, 'sentiment'] = "Positive"
  else:
    df.loc[index, 'sentiment'] = "Neutral"

In [197]:
df[df['sentiment'] == "Positive"].sort_values("wilson_lower_bound",
                                              ascending=False).head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544,0.163859,0.562259,Positive
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519,0.103870,0.516435,Positive
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139,0.212251,0.505394,Positive
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577,0.143519,0.494207,Positive
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109,0.172332,0.511282,Positive
